In [1]:
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import img_to_array
model1=load_model('model15.h5')
model2=load_model('model10.h5')

Using TensorFlow backend.


In [2]:
import imutils
import dlib
import cv2
import numpy as np
from imutils.video import FPS
from scipy.spatial import distance
from imutils import face_utils
import os

os.environ["OPENCV_VIDEOIO_PRIORITY_MSMF"] = "0"
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

In [13]:
def cnnPreprocess(img,size):
    try:
        img = cv2.resize(img, (size,size))
    except:
        return None
    else:
        img = img.astype("float") / 255.0
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)
        return img



def crop_mouth(Mouth):
        minx = (Mouth[0][0])
        maxx = (Mouth[6][0])
        miny = (min(Mouth[1:6,1]))
        maxy = (max(Mouth[7:,1]))
        mouth_rect = np.rint([minx, miny, maxx, maxy])
        mouth_rect = mouth_rect.astype(int)
        return gray[(mouth_rect[1]-20):(mouth_rect[3]+20), mouth_rect[0]-20:mouth_rect[2]+20]

def crop_eye(Eye):
        minx = (Eye[0][0])
        maxx = (Eye[3][0])
        miny = (min(Eye[1:3,1]))
        maxy = (max(Eye[4:,1]))
        
        eye_rect = np.rint([minx, miny, maxx, maxy])
        eye_rect = eye_rect.astype(int)
        return gray[(eye_rect[1]-10):(eye_rect[3]+10), eye_rect[0]-10:eye_rect[2]+10]


cap=cv2.VideoCapture(1)
flag1=flag2=0
close_counter = blinks = mem_counter= 0
blinks =0 
estate = ''
mstate=''
while True:
    ret, frame=cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray, 0)

    for subject in subjects:

        estate = ''
        mstate = ''
        shape = predict(gray, subject)
        shape = face_utils.shape_to_np(shape)#converting to NumPy Array
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        mouth = shape[mStart:mEnd]
        rroi=crop_eye(rightEye)
        lroi=crop_eye(leftEye)
        mroi=crop_mouth(mouth) 
        
        if rroi is not None and lroi is not None:

            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 225,255), 2)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 225, 255), 2)
            leftPreprocess=cnnPreprocess(rroi,24)
            rightPreprocess=cnnPreprocess(lroi,24)
            
            if leftPreprocess is not None and rightPreprocess is not None:
            
                prediction1 = (model1.predict(leftPreprocess) + model1.predict(rightPreprocess))/2.0

                if prediction1 > 0.8 :
                    estate = 'open'
                    close_counter = 0
                elif prediction1 < 0.4:
                    estate = 'close'
                    close_counter += 1

                if estate == 'open' and mem_counter >= 1:
                    blinks += 1
                    
                mem_counter = close_counter 


        
        if mroi is not None:
            mouthHull = cv2.convexHull(mouth)
            cv2.drawContours(frame, [mouthHull], -1, (0, 225,255), 2)      
            mouthPreprocess=cnnPreprocess(mroi,48)
            
            if mouthPreprocess is not None:
                prediction2 = model2.predict(mouthPreprocess) 
                
                if prediction2<0.1:
                    flag1=1
                if prediction2 > 0.4:
                    mstate='smile'                   
                    flag2=1

                    
                if flag1==1 and flag2==1:
                    mstate=''
                    flag1=0
                    flag2=0


        cv2.putText(frame, "Blinks: {}".format(blinks), (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "EState: {}".format(estate), (200, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "MState: {}".format(mstate), (400, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)


    cv2.imshow('blinks counter', frame)
    if cv2.waitKey(1)==13:
        break
    
cap.release()
cv2.destroyAllWindows()